<a href="https://colab.research.google.com/github/Shezan57/LLM/blob/main/rotten_tomatoes_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

In [4]:
data = load_dataset('rotten_tomatoes')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [5]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [6]:
data['train'][0,-1]

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'things really get weird , though not particularly scary : the movie is all portent and no content .'],
 'label': [1, 0]}

In [7]:
from transformers import pipeline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [8]:
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

In [9]:
pipe = pipeline(
    model = model_path,
    tokenizer = model_path,
    return_all_scores = True,
    device = 'cuda:0'
)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [10]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

In [11]:
y_pred = []
for output in tqdm(pipe(KeyDataset(data['test'], 'text')),
                   total = len(data['test'])):
                   negative_score = output[0]["score"]
                   positive_score = output[2]["score"]
                   assignment = np.argmax([negative_score, positive_score])
                   y_pred.append(assignment)

Disabling tokenizer parallelism, we're using DataLoader multithreading already
100%|██████████| 1066/1066 [00:18<00:00, 58.90it/s]


In [12]:
from sklearn.metrics import classification_report

In [13]:
def evaluate_performance(y_true, y_pred):
  performance = classification_report(
      y_true, y_pred,
      target_names = ['negative review', 'positive review ']
  )
  print(performance)

In [14]:
evaluate_performance(data['test']['label'], y_pred)

                  precision    recall  f1-score   support

 negative review       0.76      0.88      0.81       533
positive review        0.86      0.72      0.78       533

        accuracy                           0.80      1066
       macro avg       0.81      0.80      0.80      1066
    weighted avg       0.81      0.80      0.80      1066



DistilBERT base uncased tinetuned sst-2 model

In [15]:
pipe1 = pipeline(
    model = 'distilbert-base-uncased-finetuned-sst-2-english',
    tokenizer = 'distilbert-base-uncased-finetuned-sst-2-english',
    return_all_scores = True,
    device = 'cuda:0'
)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/714eb0fa89d2f80546fda750413ed43d93601a13/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.48.3",
  "vocab_size": 30522
}

loading configuration file config

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/714eb0fa89d2f80546fda750413ed43d93601a13/model.safetensors
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/714eb0fa89d2f80546fda750413ed43d93601a13/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.48.3",
  "vocab_size": 30522
}



vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/714eb0fa89d2f80546fda750413ed43d93601a13/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/714eb0fa89d2f80546fda750413ed43d93601a13/tokenizer_config.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/714eb0fa89d2f80546fda750413ed43d93601a13/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attenti

In [ ]:
y_pred1 = []
for output in tqdm(pipe1(KeyDataset(data['test'], 'text')),
                   total = len(data['test'])):
                   # Access the scores based on their labels
                   negative_score = next((item["score"] for item in output if item["label"] == "NEGATIVE"), 0)
                   positive_score = next((item["score"] for item in output if item["label"] == "POSITIVE"), 0)
                   assignment = np.argmax([negative_score, positive_score])
                   y_pred1.append(assignment)

 11%|█         | 112/1066 [00:00<00:06, 155.18it/s]

In [17]:
for output in tqdm(pipe1(KeyDataset(data['test'], 'text')),
                   total = len(data['test'])):
    print(output)  # Print the output to see the labels
    # ... rest of your code ...

  3%|▎         | 30/1066 [00:00<00:06, 148.17it/s]

[{'label': 'NEGATIVE', 'score': 0.00018543035548646003}, {'label': 'POSITIVE', 'score': 0.9998145699501038}]
[{'label': 'NEGATIVE', 'score': 0.00011833032476715744}, {'label': 'POSITIVE', 'score': 0.9998816251754761}]
[{'label': 'NEGATIVE', 'score': 0.9909725189208984}, {'label': 'POSITIVE', 'score': 0.009027451276779175}]
[{'label': 'NEGATIVE', 'score': 0.00012640982458833605}, {'label': 'POSITIVE', 'score': 0.9998735189437866}]
[{'label': 'NEGATIVE', 'score': 0.0004187863087281585}, {'label': 'POSITIVE', 'score': 0.9995811581611633}]
[{'label': 'NEGATIVE', 'score': 0.0007528371643275023}, {'label': 'POSITIVE', 'score': 0.9992471933364868}]
[{'label': 'NEGATIVE', 'score': 0.00020422377565409988}, {'label': 'POSITIVE', 'score': 0.9997957348823547}]
[{'label': 'NEGATIVE', 'score': 0.00033533736132085323}, {'label': 'POSITIVE', 'score': 0.9996646642684937}]
[{'label': 'NEGATIVE', 'score': 0.00018026246107183397}, {'label': 'POSITIVE', 'score': 0.999819815158844}]
[{'label': 'NEGATIVE', '

  6%|▌         | 60/1066 [00:00<00:06, 147.05it/s]

[{'label': 'NEGATIVE', 'score': 0.00016088357369881123}, {'label': 'POSITIVE', 'score': 0.9998390674591064}]
[{'label': 'NEGATIVE', 'score': 0.9993811845779419}, {'label': 'POSITIVE', 'score': 0.0006187416729517281}]
[{'label': 'NEGATIVE', 'score': 0.00012532198161352426}, {'label': 'POSITIVE', 'score': 0.9998747110366821}]
[{'label': 'NEGATIVE', 'score': 0.000198428489966318}, {'label': 'POSITIVE', 'score': 0.9998015761375427}]
[{'label': 'NEGATIVE', 'score': 0.0003970210091210902}, {'label': 'POSITIVE', 'score': 0.9996029734611511}]
[{'label': 'NEGATIVE', 'score': 0.000257931649684906}, {'label': 'POSITIVE', 'score': 0.9997420907020569}]
[{'label': 'NEGATIVE', 'score': 0.006819942034780979}, {'label': 'POSITIVE', 'score': 0.9931800365447998}]
[{'label': 'NEGATIVE', 'score': 0.00012404413428157568}, {'label': 'POSITIVE', 'score': 0.9998759031295776}]
[{'label': 'NEGATIVE', 'score': 0.00023928408336360008}, {'label': 'POSITIVE', 'score': 0.9997606873512268}]
[{'label': 'NEGATIVE', 'sco

  9%|▊         | 91/1066 [00:00<00:06, 149.56it/s]

...

[{'label': 'NEGATIVE', 'score': 0.9996138215065002}, {'label': 'POSITIVE', 'score': 0.0003861985169351101}]
[{'label': 'NEGATIVE', 'score': 0.9978390336036682}, {'label': 'POSITIVE', 'score': 0.0021609992254525423}]
[{'label': 'NEGATIVE', 'score': 0.9998099207878113}, {'label': 'POSITIVE', 'score': 0.00019004927889909595}]
[{'label': 'NEGATIVE', 'score': 0.999656081199646}, {'label': 'POSITIVE', 'score': 0.00034397587296552956}]
[{'label': 'NEGATIVE', 'score': 0.0009655263274908066}, {'label': 'POSITIVE', 'score': 0.9990345239639282}]
[{'label': 'NEGATIVE', 'score': 0.999636173248291}, {'label': 'POSITIVE', 'score': 0.00036384386476129293}]
[{'label': 'NEGATIVE', 'score': 0.999755322933197}, {'label': 'POSITIVE', 'score': 0.0002446672588121146}]
[{'label': 'NEGATIVE', 'score': 0.9931374788284302}, {'label': 'POSITIVE', 'score': 0.006862539332360029}]
[{'label': 'NEGATIVE', 'score': 0.9996517896652222}, {'label': 'POSITIVE', 'score': 0.0003481867315713316}]
[{'label': 'NEGATIVE', 'score

 98%|█████████▊| 1040/1066 [00:06<00:00, 79.63it/s]

[{'label': 'NEGATIVE', 'score': 0.9996761083602905}, {'label': 'POSITIVE', 'score': 0.0003238585777580738}]
[{'label': 'NEGATIVE', 'score': 0.0030582279432564974}, {'label': 'POSITIVE', 'score': 0.9969417452812195}]
[{'label': 'NEGATIVE', 'score': 0.999771773815155}, {'label': 'POSITIVE', 'score': 0.00022823232575319707}]
[{'label': 'NEGATIVE', 'score': 0.9985188841819763}, {'label': 'POSITIVE', 'score': 0.001481099403463304}]
[{'label': 'NEGATIVE', 'score': 0.9980753660202026}, {'label': 'POSITIVE', 'score': 0.0019245753064751625}]
[{'label': 'NEGATIVE', 'score': 0.9899499416351318}, {'label': 'POSITIVE', 'score': 0.01005004346370697}]
[{'label': 'NEGATIVE', 'score': 0.9991414546966553}, {'label': 'POSITIVE', 'score': 0.0008585307514294982}]
[{'label': 'NEGATIVE', 'score': 0.9996572732925415}, {'label': 'POSITIVE', 'score': 0.0003427041810937226}]
[{'label': 'NEGATIVE', 'score': 0.999784529209137}, {'label': 'POSITIVE', 'score': 0.00021545993513427675}]
[{'label': 'NEGATIVE', 'score':

100%|██████████| 1066/1066 [00:07<00:00, 151.22it/s]

[{'label': 'NEGATIVE', 'score': 0.9983950257301331}, {'label': 'POSITIVE', 'score': 0.001604996738024056}]
[{'label': 'NEGATIVE', 'score': 0.9985937476158142}, {'label': 'POSITIVE', 'score': 0.001406251103617251}]
[{'label': 'NEGATIVE', 'score': 0.9997954964637756}, {'label': 'POSITIVE', 'score': 0.00020446265989448875}]
[{'label': 'NEGATIVE', 'score': 0.9989113807678223}, {'label': 'POSITIVE', 'score': 0.0010886803502216935}]
[{'label': 'NEGATIVE', 'score': 0.999721348285675}, {'label': 'POSITIVE', 'score': 0.000278667313978076}]
[{'label': 'NEGATIVE', 'score': 0.9994284510612488}, {'label': 'POSITIVE', 'score': 0.0005714856088161469}]
[{'label': 'NEGATIVE', 'score': 0.9995303153991699}, {'label': 'POSITIVE', 'score': 0.0004696497635450214}]
[{'label': 'NEGATIVE', 'score': 0.9996867179870605}, {'label': 'POSITIVE', 'score': 0.00031326714088208973}]
[{'label': 'NEGATIVE', 'score': 0.9995388984680176}, {'label': 'POSITIVE', 'score': 0.0004611751064658165}]
[{'label': 'NEGATIVE', 'score'

In [ ]:
evalueate_performance(data['test']['label'], y_pred1)